In [ ]:
from pymc import Uniform, deterministic, MCMC, Binomial, Normal, SkewNormal
from pymc import stochastic, graph, Model, DiscreteUniform, binomial_like
import numpy as np
import pandas as pd
from pylab import hist, show, normpdf
import matplotlib.pyplot as plt
import math
import toolboxes 
import DataAndMetadataCats as dmc
from pymc.Matplot import plot
from scipy import stats

In [ ]:
directorio_dp = ('../../inst/extdata/erradicaciones-mamiferos/')
recurso = 'captura_gatos.csv'
DatosSocorro = dmc.DataAndMetadataCats(directorio_dp,recurso)

In [ ]:
n_datos = len(DatosSocorro.getData())
nombre_esfuerzo = DatosSocorro.glosario["esfuerzo"]
nombre_capturas = DatosSocorro.glosario["capturas"]

Creo que aquí se tiene que hacer un cambio de unidades en el esfuerzo

In [ ]:
esfuerzo = np.array(DatosSocorro.getValue(nombre_esfuerzo)/(30 * 7 * 5))  #Días hombre
capturas = np.array(DatosSocorro.getValue(nombre_capturas))
print(sum(capturas))

En una segunda campaña se pudieron capturar:

In [ ]:
esfuerzo2 = np.array(DatosSocorro.getValue(nombre_esfuerzo)/(30 * 7 * 5))  #Días hombre
capturas2 = np.array(DatosSocorro.getValue(nombre_capturas))

## El modelo

El parámetro $\tau$ se define como: $$\tau = \frac{1}{\sigma^2}.$$

## Muestreos Monte-Carlo

In [ ]:
Modelo_gatitos = Model(dmc.metodo_Ramsey(esfuerzo, capturas))
for iIteraciones in range(0,100):
    Modelo_gatitos = MCMC(dmc.metodo_Ramsey(esfuerzo, capturas))
    Modelo_gatitos.sample(iter = 6000000, burn = 3000000)
    a = pd.Series(Modelo_gatitos.trace('a_captura')[:])
    b = pd.Series(Modelo_gatitos.trace('b_captura')[:])
    No = pd.Series(Modelo_gatitos.trace('N_o')[:])
    pd.DataFrame({"a":a,"b":b,"No":No}).to_csv(f'../../resultados/muestras_normal_datos_completos_{iIteraciones}.csv')

## Histogramas de las distribuciones posteriores

### Salvado de los resultados

In [ ]:
plt.plot(No)

La cantidad de gatos que quedan son:

In [ ]:
remanentes = np.percentile(Modelo_gatitos.trace('N_o')[:],[0.5,95.5]) - sum(capturas)
print(remanentes)

In [ ]:
histograma_No = hist(No, bins=1000)
show()

## Las distribuciones posteriores de $\alpha$ y $\beta$

### ¿Cuáles son los valores de los parámetros de las distribuciones posteriores?¶

Para $\alpha$, la media y la desviación estándar son:

In [ ]:
alpha_media = np.mean(a)
alpha_desviacion = np.std(a)
print(alpha_media,alpha_desviacion)

Para $\beta$, la media y la desviación estándar son:

In [ ]:
beta_media = np.mean(b)
beta_desviacion = np.std(b)
print(beta_media, beta_desviacion)

### Graficado de las distribuciones posteriores de $\alpha$ y $\beta$

In [ ]:
n, bins, patches = hist(a, bins=1000, normed = 1)
y = normpdf( bins, alpha_media,alpha_desviacion)
l = plt.plot(bins, y, 'r--', linewidth=1)
show()

In [ ]:
n, bins, patches = hist(b, bins=1000, normed = 1)
y = normpdf( bins, beta_media,beta_desviacion)
l = plt.plot(bins, y, 'r--', linewidth=1)
show()

### Normalidad de los datos $\alpha$ y $\beta$

In [ ]:
n_ciclos = 100
p_valor = []
for iCiclo in range(n_ciclos):
    x = pd.Series(a).sample(n=5000)
    p_valor.append(stats.shapiro(x)[1])
sum(np.array(p_valor) > 0.05) * 100 / n_ciclos